In [ ]:
import pandas as pd
import numpy as np

import json
import pickle

from fastembed import TextEmbedding, SparseTextEmbedding

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, SparseVectorParams 
from qdrant_client.models import Distance, VectorParams, PointStruct
from qdrant_client.models import Filter, FieldCondition, MatchValue
from qdrant_client.http.exceptions import UnexpectedResponse

from tqdm import tqdm

In [2]:
df = pd.read_json(r"E:\SUDO_FINAL\RAG2\app\data\local_description_final.jsonl", lines=True)
df["region"] = df["id"].str.extract(r'([A-Z])$')
region_map = {"B": "miền Bắc", "T": "miền Trung", "N": "miền Nam"}
df["region"] = df["region"].map(region_map)
df["region"] = df["region"].where(df["region"].notna(), None)


def build_full_description(row):
    desc = row["description"] if pd.notna(row["description"]) else ""
    region = f" ({row['region']})" if pd.notna(row["region"]) else ""
    return f"Ký hiệu của từ '{row['word']}'{region}: {desc}"

df["full_description"] = df.apply(build_full_description, axis=1)

df

,id,word,video_url,description,region,full_description
0,D0005T,Miến Điện (nước Mi-an-ma),https://storage.googleapis.com/vsl-data-bucket...,"Tay phải giống số tự nhiên ""1"", lòng bàn tay h...",miền Trung,Ký hiệu của từ 'Miến Điện (nước Mi-an-ma)' (mi...
1,D0005B,Miến Điện (nước Mi-an-ma),https://storage.googleapis.com/vsl-data-bucket...,"Tay phải giống số tự nhiên ""1"", lòng bàn tay h...",miền Bắc,Ký hiệu của từ 'Miến Điện (nước Mi-an-ma)' (mi...
2,D0005N,Miến Điện (nước Mi-an-ma),https://storage.googleapis.com/vsl-data-bucket...,"Tay phải giống số tự nhiên ""1"", lòng bàn tay h...",miền Nam,Ký hiệu của từ 'Miến Điện (nước Mi-an-ma)' (mi...
3,D0001T,địa chỉ,https://storage.googleapis.com/vsl-data-bucket...,"1. Tay phải nắm, ngón trỏ duỗi thẳng, các ngón...",miền Trung,Ký hiệu của từ 'địa chỉ' (miền Trung): 1. Tay ...
4,D0001B,địa chỉ,https://storage.googleapis.com/vsl-data-bucket...,"Tay phải nắm, ngón trỏ duỗi thẳng, các ngón cò...",miền Bắc,Ký hiệu của từ 'địa chỉ' (miền Bắc): Tay phải ...
...,...,...,...,...,...,...
4357,W03938B,xanh xao,https://storage.googleapis.com/vsl-data-bucket...,"Tay phải mở, các ngón tay khép lại, lòng bàn t...",miền Bắc,Ký hiệu của từ 'xanh xao' (miền Bắc): Tay phải...
4358,W03938N,xanh xao,https://storage.googleapis.com/vsl-data-bucket...,"Tay phải mở, các ngón tay khép lại, lòng bàn t...",miền Nam,Ký hiệu của từ 'xanh xao' (miền Nam): Tay phải...
4359,W03967N,xe khách,https://storage.googleapis.com/vsl-data-bucket...,"1. Hai tay nắm, ngón cái và ngón trỏ mở ra, lò...",miền Nam,Ký hiệu của từ 'xe khách' (miền Nam): 1. Hai t...
4360,W03978,xếp,https://storage.googleapis.com/vsl-data-bucket...,"Hai tay khép, lòng bàn tay hướng vào nhau. Tay...",None,"Ký hiệu của từ 'xếp': Hai tay khép, lòng bàn t..."


In [10]:
# Dense model (semantic)
dense_model = TextEmbedding("intfloat/multilingual-e5-large") #dense_model = TextEmbedding("BAAI/bge-m3")

# Sparse model (lexical)
sparse_model = SparseTextEmbedding("Qdrant/bm25")

C:\Users\ThienLaptop\AppData\Local\Temp\ipykernel_8600\1482264275.py:2: UserWarning: The model intfloat/multilingual-e5-large now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  dense_model = TextEmbedding("intfloat/multilingual-e5-large") #dense_model = TextEmbedding("BAAI/bge-m3")


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [16]:
dense_embeddings = np.array(list(dense_model.embed(df["full_description"].tolist()))).astype("float32")
sparse_embeddings = list(sparse_model.embed(df["full_description"].tolist()))

print("Dense embeddings shape:", dense_embeddings.shape)
print("Sparse embeddings count:", len(sparse_embeddings))

Dense embeddings shape: (4362, 1024)
Sparse embeddings count: 4362


In [37]:
# Save dense embeddings
np.save("dense_embeddings.npy", dense_embeddings)

# Save sparse embeddings
with open("sparse_embeddings.pkl", "wb") as f:
    pickle.dump(sparse_embeddings, f)

print("Dense embeddings shape:", dense_embeddings.shape)
print("Sparse embeddings count:", len(sparse_embeddings))

Dense embeddings shape: (4362, 1024)
Sparse embeddings count: 4362


In [7]:
client_qdrant = QdrantClient(path="hybrid_search_qdrant_storage")    

In [44]:
from qdrant_client.models import VectorParams, SparseVectorParams, Distance

client_qdrant.recreate_collection(
    collection_name="sign_vectors",
    vectors_config={
        "dense": VectorParams(size=dense_embeddings.shape[1], distance=Distance.COSINE)
    },
    sparse_vectors_config={
        "sparse": SparseVectorParams()
    }
)

C:\Users\ThienLaptop\AppData\Local\Temp\ipykernel_8600\4290633075.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client_qdrant.recreate_collection(


True

In [ ]:
points = []
batch_size = 500 

for i in tqdm(range(len(df)), desc="Building hybrid vectors"):
    sparse_vec = sparse_embeddings[i]

    if hasattr(sparse_vec, "indices"):
        sparse_data = SparseVector(
            indices=sparse_vec.indices.tolist(),
            values=sparse_vec.values.tolist()
        )
    elif isinstance(sparse_vec, dict):
        sparse_data = SparseVector(
            indices=sparse_vec["indices"],
            values=sparse_vec["values"]
        )
    else:
        raise TypeError(f"Unexpected sparse embedding type: {type(sparse_vec)}")

    points.append(
        PointStruct(
            id=i,
            vector={
                "dense": dense_embeddings[i].tolist(),
                "sparse": sparse_data
            },
            payload={
                "word": df.iloc[i]["word"],
                "region": df.iloc[i]["region"],
                "video": df.iloc[i]["video_url"],
                "description": df.iloc[i]["description"],
                "full_description": df.iloc[i]["full_description"]
            }
        )
    )

    if len(points) >= batch_size or i == len(df) - 1:
        client_qdrant.upsert(collection_name="sign_vectors", points=points)
        points = [] 

print("All hybrid points upserted successfully.")

Building hybrid vectors: 100%|██████████| 4362/4362 [16:50<00:00,  4.32it/s] 

✅ All hybrid points upserted successfully.


In [ ]:
from collections import defaultdict

def hybrid_search(query: str, top_k: int = 5, alpha: float = 0.6):
    """Hybrid search: dense + sparse cho Qdrant local"""
    dense_query = list(dense_model.embed([query]))[0]
    sparse_query = list(sparse_model.embed([query]))[0]

    collection = client_qdrant._client.collections["sign_vectors"]

    # --- Dense search ---
    dense_results = collection.search(
        query_vector=dense_query,
        vector_name="dense",
        limit=top_k * 3
    )

    # --- Sparse search ---
    sparse_results = collection.search(
        query_vector=sparse_query["values"],
        vector_name="sparse",
        limit=top_k * 3
    )

    # --- Combine ---
    scores = defaultdict(float)
    for r in dense_results:
        scores[r.id] += alpha * r.score
    for r in sparse_results:
        scores[r.id] += (1 - alpha) * r.score

    final_results = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

    print(f"\n Hybrid Search cho truy vấn: '{query}' (α={alpha})")
    print("-" * 80)
    for rank, (pid, score) in enumerate(final_results, 1):
        payload = client_qdrant.retrieve(collection_name="sign_vectors", ids=[pid])[0].payload
        print(f"{rank}. {payload['word']} ({payload['region']})")
        print(f"    {payload['description']}")
        print(f"    Score: {score:.4f}\n")

# ------------------------------------------------------------
#  Test
hybrid_search("trái cây miền Nam", alpha=0.6)


TypeError: LocalCollection.search() got an unexpected keyword argument 'vector_name'